In [ ]:
import sys
import os
import cv2
import mysql.connector
import numpy as np

from PyQt5.QtWidgets import QApplication, QDialog, QPushButton, QLabel, QLineEdit, QMessageBox
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

class Dialog(QDialog):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(0, 0, 663, 462)
        self.setWindowTitle("Dialog")

        # Face Recognition Button
        self.pushButton_3 = QPushButton("Face Recognition", self)
        self.pushButton_3.setGeometry(170, 370, 331, 71)
        self.pushButton_3.setStyleSheet("font-size: 12pt;")
        self.pushButton_3.clicked.connect(self.face_recognition)

        # MSSV Label and LineEdit
        self.label = QLabel("MSSV", self)
        self.label.setGeometry(20, 20, 47, 21)
        self.label.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit = QLineEdit(self)
        self.lineEdit.setGeometry(100, 20, 201, 21)
        self.lineEdit.setStyleSheet("font-size: 12pt;")
        
        # Class Label and LineEdit
        self.label_2 = QLabel("Mã lớp:", self)
        self.label_2.setGeometry(20, 70, 47, 21)
        self.label_2.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_2 = QLineEdit(self)
        self.lineEdit_2.setGeometry(100, 70, 201, 21)
        self.lineEdit_2.setStyleSheet("font-size: 12pt;")

        # Username Label and LineEdit
        self.label_3 = QLabel("Username", self)
        self.label_3.setGeometry(20, 120, 71, 21)
        self.label_3.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_3 = QLineEdit(self)
        self.lineEdit_3.setGeometry(100, 120, 201, 21)
        self.lineEdit_3.setStyleSheet("font-size: 12pt;")

        # Sex Label and LineEdit
        self.label_4 = QLabel("Sex", self)
        self.label_4.setGeometry(20, 170, 47, 21)
        self.label_4.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_4 = QLineEdit(self)
        self.lineEdit_4.setGeometry(100, 170, 201, 21)
        self.lineEdit_4.setStyleSheet("font-size: 12pt;")

        # Dob Label and LineEdit
        self.label_5 = QLabel("Dob", self)
        self.label_5.setGeometry(350, 20, 47, 21)
        self.label_5.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_5 = QLineEdit(self)
        self.lineEdit_5.setGeometry(420, 20, 201, 21)
        self.lineEdit_5.setStyleSheet("font-size: 12pt;")

        # Address Label and LineEdit
        self.label_6 = QLabel("Address", self)
        self.label_6.setGeometry(350, 70, 61, 21)
        self.label_6.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_6 = QLineEdit(self)
        self.lineEdit_6.setGeometry(420, 70, 201, 21)
        self.lineEdit_6.setStyleSheet("font-size: 12pt;")

        # Phone Label and LineEdit
        self.label_7 = QLabel("Phone", self)
        self.label_7.setGeometry(350, 120, 47, 21)
        self.label_7.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_7 = QLineEdit(self)
        self.lineEdit_7.setGeometry(420, 120, 201, 21)
        self.lineEdit_7.setStyleSheet("font-size: 12pt;")

        # Save to DB and Generate Dataset Button
        self.pushButton_6 = QPushButton("Save to DB and generate dataset", self)
        self.pushButton_6.setGeometry(350, 160, 271, 51)
        self.pushButton_6.setStyleSheet("font-size: 12pt;")
        self.pushButton_6.clicked.connect(self.save_to_db_and_generate_dataset)

        # MSSV for Training Label and LineEdit
        self.label_8 = QLabel("Nhập MSSV cần train:", self)
        self.label_8.setGeometry(100, 230, 161, 21)
        self.label_8.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_8 = QLineEdit(self)
        self.lineEdit_8.setGeometry(80, 260, 201, 21)
        self.lineEdit_8.setStyleSheet("font-size: 12pt;")

        # Training AI Button
        self.pushButton_5 = QPushButton("Training AI", self)
        self.pushButton_5.setGeometry(110, 290, 151, 51)
        self.pushButton_5.setStyleSheet("font-size: 12pt;")
        self.pushButton_5.clicked.connect(self.train_ai)

        # Generate Dataset Button
        self.pushButton_4 = QPushButton("Generate dataset", self)
        self.pushButton_4.setGeometry(400, 290, 171, 51)
        self.pushButton_4.setStyleSheet("font-size: 12pt;")
        self.pushButton_4.clicked.connect(self.generate_dataset)

        # Recreate Dataset MSSV Label and LineEdit
        self.label_9 = QLabel("Nhập MSSV cần tạo lại dataset:", self)
        self.label_9.setGeometry(370, 230, 231, 21)
        self.label_9.setStyleSheet("font-size: 12pt;")
        
        self.lineEdit_9 = QLineEdit(self)
        self.lineEdit_9.setGeometry(390, 260, 201, 21)
        self.lineEdit_9.setStyleSheet("font-size: 12pt;")

    # Methods for button functionality
    def save_to_db_and_generate_dataset(self):
        mssv = self.lineEdit.text()
        ma_lop = self.lineEdit_2.text()
        username = self.lineEdit_3.text()
        sex = self.lineEdit_4.text()
        dob = self.lineEdit_5.text()
        address = self.lineEdit_6.text()
        phone = self.lineEdit_7.text()
    
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='12345',
            database='face_recognitiondb'
        )
        cursor = connection.cursor()
    
        try:
            # Thêm thông tin cá nhân vào cơ sở dữ liệu
            insert_query = """
            INSERT INTO Personal_Info (MSSV, Ma_Lop, Username, Sex, Dob, Address, Phone) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, (mssv, ma_lop, username, sex, dob, address, phone))
            connection.commit()
    
            # Tạo thư mục cho ảnh
            dataset_path = '../Input/Dataset'
            person_folder = os.path.join(dataset_path, mssv)
            if not os.path.exists(person_folder):
                os.makedirs(person_folder)
    
            # Bắt đầu lấy ảnh từ webcam
            cap = cv2.VideoCapture(0)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
            num_images = 100
            count = 0
    
            while count < num_images:
                ret, frame = cap.read()
                if not ret:
                    continue
    
                frame = cv2.flip(frame, 1)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
                # Kiểm tra xem có khuôn mặt nào không
                if len(faces) > 0:
                    for (x, y, w, h) in faces:
                        count += 1
                        face = gray[y:y + h, x:x + w]
                        face = cv2.resize(face, (200, 200))
    
                        image_path = os.path.join(person_folder, f'face_{count}.jpg')
                        cv2.imwrite(image_path, face)
    
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
                cv2.imshow('Frame', frame)
    
                if cv2.waitKey(1) & 0xFF == ord('q') or count >= num_images:
                    break
    
            cap.release()
            cv2.destroyAllWindows()
    
            # Lưu thông tin hình ảnh vào cơ sở dữ liệu
            for i in range(1, count + 1):
                image_path = os.path.join(person_folder, f'face_{i}.jpg').replace("\\", "/")
                cursor.execute("INSERT INTO Image (MSSV, URL) VALUES (%s, %s)", (mssv, image_path))
    
            connection.commit()
            QMessageBox.information(self, "Success", "Data saved successfully and images generated.")
    
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "Database Error", f"Error: {err}")
        finally:
            cursor.close()
            connection.close()
    
    
        def train_ai(self):
            mssv = self.lineEdit_8.text()
    
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='12345',
                database='face_recognitiondb'
            )
            cursor = connection.cursor()
    
            try:
                cursor.execute("SELECT URL FROM Image WHERE MSSV = %s", (mssv,))
                images = cursor.fetchall()
    
                if not images:
                    QMessageBox.warning(self, "Warning", "No images found for this MSSV.")
                    return
    
                dataset_path = '../Input/Dataset/' + mssv
                faces = []
    
                for image_path in images:
                    img = cv2.imread(image_path[0], cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        faces.append(img.flatten())
    
                faces = np.array(faces)
    
                scaler = StandardScaler()
                faces_scaled = scaler.fit_transform(faces)
    
                pca = PCA(n_components=10)
                eigenfaces = pca.fit_transform(faces_scaled)
    
                model_filename = f'{mssv}_eigenfaces_model.npy'
                np.save(model_filename, eigenfaces)
    
                QMessageBox.information(self, "Success", "Model trained successfully and saved.")
    
            except mysql.connector.Error as err:
                QMessageBox.critical(self, "Database Error", f"Error: {err}")
            finally:
                cursor.close()
                connection.close()
            
    def generate_dataset(self):
        mssv = self.lineEdit_9.text()  # Lấy MSSV từ input
    
        if not mssv:
            QMessageBox.warning(self, "Warning", "Please enter an MSSV.")
            return
    
        # Kết nối cơ sở dữ liệu
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='12345',
            database='face_recognitiondb'
        )
        cursor = connection.cursor()
    
        try:
            # Kiểm tra nếu MSSV có tồn tại trong bảng Personal_Info
            cursor.execute("SELECT * FROM Personal_Info WHERE MSSV = %s", (mssv,))
            person = cursor.fetchone()
    
            if not person:
                QMessageBox.warning(self, "Warning", "No person found with this MSSV.")
                return
    
            # Tạo thư mục cho MSSV nếu chưa có
            dataset_path = '../Input/Dataset'
            person_folder = os.path.join(dataset_path, mssv)
            if not os.path.exists(person_folder):
                os.makedirs(person_folder)
    
            # Bắt đầu lấy ảnh từ webcam
            cap = cv2.VideoCapture(0)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
            num_images = 100  # Số lượng ảnh cần chụp
            count = 0
    
            while count < num_images:
                ret, frame = cap.read()
                if not ret:
                    continue
    
                frame = cv2.flip(frame, 1)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
                # Phát hiện khuôn mặt
                if len(faces) > 0:
                    for (x, y, w, h) in faces:
                        count += 1
                        face = gray[y:y + h, x:x + w]
                        face = cv2.resize(face, (200, 200))
    
                        # Lưu ảnh vào thư mục
                        image_path = os.path.join(person_folder, f'face_{count}.jpg')
                        cv2.imwrite(image_path, face)
    
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
                cv2.imshow('Frame', frame)
    
                # Thoát khi nhấn 'q' hoặc chụp đủ số ảnh
                if cv2.waitKey(1) & 0xFF == ord('q') or count >= num_images:
                    break
    
            cap.release()
            cv2.destroyAllWindows()
    
            # Lưu thông tin hình ảnh vào cơ sở dữ liệu
            for i in range(1, count + 1):
                image_path = os.path.join(person_folder, f'face_{i}.jpg').replace("\\", "/")
                cursor.execute("INSERT INTO Image (MSSV, URL) VALUES (%s, %s)", (mssv, image_path))
    
            connection.commit()
            QMessageBox.information(self, "Success", "Dataset generated successfully.")
    
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "Database Error", f"Error: {err}")
        finally:
            cursor.close()
            connection.close()

    def train_ai(self):
        mssv = self.lineEdit_8.text()
    
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='12345',
            database='face_recognitiondb'
        )
        cursor = connection.cursor()
    
        try:
            cursor.execute("SELECT URL FROM Image WHERE MSSV = %s", (mssv,))
            images = cursor.fetchall()
    
            if not images:
                QMessageBox.warning(self, "Warning", "No images found for this MSSV.")
                return
    
            faces = []
    
            for image_path in images:
                img = cv2.imread(image_path[0], cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img_resized = cv2.resize(img, (200, 200))
                    faces.append(img_resized.flatten())
    
            faces = np.array(faces)
    
            # Standardize the data
            scaler = StandardScaler()
            faces_scaled = scaler.fit_transform(faces)
    
            # Train PCA
            pca = PCA(n_components=min(10, faces_scaled.shape[0]-1))
            eigenfaces = pca.fit_transform(faces_scaled)
    
            # Save the models
            model_dir = f'../Input/Model/'
            np.save(os.path.join(model_dir, f'{mssv}_eigenfaces_model.npy'), eigenfaces)
            np.save(os.path.join(model_dir, f'{mssv}_pca_model.npy'), pca)  # Save PCA model
            np.save(os.path.join(model_dir, f'{mssv}_scaler_model.npy'), scaler)  # Save scaler model
    
            QMessageBox.information(self, "Success", "Model trained successfully and saved.")
    
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "Database Error", f"Error: {err}")
        finally:
            cursor.close()
            connection.close()
            
    def face_recognition(self):
        # Bắt đầu webcam
        cap = cv2.VideoCapture(0)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
        # Tải tất cả các mô hình đã huấn luyện và MSSV tương ứng
        models = {}
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='12345',
            database='face_recognitiondb'
        )
        cursor = connection.cursor()
    
        try:
            cursor.execute("SELECT MSSV FROM Personal_Info")
            mssvs = cursor.fetchall()
    
            for mssv in mssvs:
                model_filename = f'../Input/Model/{mssv[0]}_eigenfaces_model.npy'  # Cập nhật đường dẫn
                pca_filename = f'../Input/Model/{mssv[0]}_pca_model.npy'  # Cập nhật đường dẫn
                if os.path.exists(model_filename) and os.path.exists(pca_filename):
                    eigenfaces = np.load(model_filename)
                    pca = np.load(pca_filename, allow_pickle=True).item()  # Tải mô hình PCA
                    models[mssv[0]] = (eigenfaces, pca)
    
        except mysql.connector.Error as err:
            QMessageBox.critical(self, "Database Error", f"Error: {err}")
            return
        finally:
            cursor.close()
            connection.close()
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            # Flip the frame horizontally to correct the camera feed
            frame = cv2.flip(frame, 1)  # Flip horizontally
    
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
            for (x, y, w, h) in faces:
                face = gray[y:y + h, x:x + w]
                face_resized = cv2.resize(face, (200, 200)).flatten()  # Resize and flatten the face
                face_scaled = StandardScaler().fit_transform([face_resized])  # Scale the face
    
                matched_mssv = None
                closest_distance = float('inf')
    
                for mssv, (eigenfaces, pca) in models.items():
                    n_samples = eigenfaces.shape[0]
    
                    # Kiểm tra số lượng mẫu
                    if n_samples >= 2:  # Cần ít nhất hai mẫu để thực hiện PCA
                        # Chuyển đổi khuôn mặt hiện tại
                        try:
                            face_eigenface = pca.transform(face_scaled)  # Chuyển đổi khuôn mặt
                        except ValueError as e:
                            print(f"Error transforming face for MSSV {mssv}: {e}")
                            continue
    
                        # Tính khoảng cách
                        distances = euclidean_distances(face_eigenface, eigenfaces)
                        min_distance = np.min(distances)
    
                        if min_distance < closest_distance:
                            closest_distance = min_distance
                            matched_mssv = mssv
    
                # Hiển thị kết quả nhận diện
                if matched_mssv is not None:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    cv2.putText(frame, f'MSSV: {matched_mssv}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
                    # Lấy và hiển thị thông tin người dùng
                    connection = mysql.connector.connect(
                        host='localhost',
                        user='root',
                        password='12345',
                        database='face_recognitiondb'
                    )
                    cursor = connection.cursor()
                    
                    try:
                        # Kết nối đến cơ sở dữ liệu
                        connection = mysql.connector.connect(
                            host='localhost',
                            user='root',
                            password='12345',
                            database='face_recognitiondb'
                        )
                        cursor = connection.cursor()
                    
                        # Gọi stored procedure với MSSV
                        cursor.callproc('sp_GetStudentInfo', (matched_mssv,))
                        
                        # Lấy kết quả từ stored procedure
                        for result in cursor.stored_results():
                            user_info = result.fetchone()  # Lấy kết quả đầu tiên
                    
                        if user_info:
                            # Hiển thị thông tin người dùng
                            username_text = f'Username: {user_info[0]}'
                            class_text = f'Ma lop: {user_info[1]}'
                            department_text = f'Khoa: {user_info[2]}'
                            
                            # Vẽ từng dòng thông tin
                            cv2.putText(frame, username_text, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
                            cv2.putText(frame, class_text, (x, y + h + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
                            cv2.putText(frame, department_text, (x, y + h + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)

                    
                    except mysql.connector.Error as err:
                        print(f"Error retrieving user info: {err}")
                    finally:
                        cursor.close()
                        connection.close()
    
            cv2.imshow('Face Recognition', frame)
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
        cap.release()
        cv2.destroyAllWindows()        

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    dialog = Dialog()
    dialog.show()
    sys.exit(app.exec_())